In [1]:
from carla import Benchmark
from IPython.display import display
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [ ]:
scm = CausalModel("credit")

In [ ]:
scm.cgm.draw()

In [2]:

dataset = scm.generate_dataset(1000)
dataset.df.to_csv('credit_synthetic.csv',index=False)
display(dataset.df)

,label,x1,x2,x3,x4,x6,x5,x7
0,0.0,1.0,-14.433597,-0.180979,0.682516,-2.539147,4.427678,1.821287
1,0.0,0.0,-17.807789,-0.145061,-1.269254,-2.270610,-5.521828,-0.777808
2,0.0,0.0,-0.497926,-0.097943,-1.283408,1.810407,-9.423442,-1.605173
3,0.0,1.0,-11.752795,-0.102697,-1.842651,-1.427213,-2.300992,-5.285491
4,1.0,1.0,-3.339395,0.021847,2.944004,3.583888,8.928894,-5.761631
...,...,...,...,...,...,...,...,...
995,1.0,0.0,5.306878,-0.023677,0.204469,-1.702920,-5.438995,0.285482
996,0.0,0.0,-9.411433,-0.331767,-3.608741,-2.421001,-9.262225,0.624211
997,1.0,1.0,1.942773,0.264792,2.284038,3.300092,3.781434,1.608431
998,0.0,0.0,14.233095,0.077207,2.188335,1.707255,8.720171,-7.036546


In [3]:
'''Build Dataset for Wachter'''
import pandas as pd
# Load Data 
dataframe = pd.read_csv('./credit_synthetic.csv')
continuous = dataframe.drop(columns=['label']).columns

dataset = CsvCatalog(file_path="credit_synthetic.csv",
                     continuous=continuous,
                     categorical=[],
                     immutables=[],
                     target='label')

In [4]:
#Model 

training_params = {"lr": 0.01, "epochs": 3, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.47333333333333333, balance on test set 0.492
Epoch 0/2
----------
train Loss: 0.7362 Acc: 0.4547

test Loss: 0.6934 Acc: 0.5240

Epoch 1/2
----------
train Loss: 0.6745 Acc: 0.6027

test Loss: 0.6249 Acc: 0.7040

Epoch 2/2
----------
train Loss: 0.5652 Acc: 0.7360

test Loss: 0.4687 Acc: 0.8320



In [5]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:2]

display(test_factual)

,label,x1,x2,x3,x4,x6,x5,x7
0,0.0,1.0,0.164208,0.244390,0.682901,0.377056,0.715439,0.579422
1,0.0,0.0,0.114204,0.294977,0.590889,0.394369,0.378143,0.507552


In [6]:
#hyperparams = {"loss_type": "BCE"}

#recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
#cfs = recourse_method.get_counterfactuals(test_factual)

#from carla.models.negative_instances import predict_negative_instances
from carla.recourse_methods.catalog.causal_recourse import (
    CausalRecourse,
    constraints,
    samplers,
)

hyperparams = {
    "optimization_approach": "brute_force",
    "num_samples": 10,
    "scm": scm,
    "constraint_handle": constraints.point_constraint,
    "sampler_handle": samplers.sample_true_m0,
}
cfs = CausalRecourse(ml_model, hyperparams).get_counterfactuals(test_factual)

display(cfs)

Initialization Starting
Initialization Finished
Start get CF
index 0
1
2
3
4
5
node x1
node x2
node x3
node x4
node x5
node x6
node x7
Possible Actions [[-0.498, -0.27578, -0.05356, 0.16867, 0.39089, 0.61311, 0.83533, 1.05756, 1.27978, 1.502, None], [-0.37993, -0.1577, 0.06452, 0.28674, 0.50896, 0.73118, 0.95341, 1.17563, 1.39785, 1.62007, None], [-0.41645, -0.19423, 0.02799, 0.25021, 0.47243, 0.69466, 0.91688, 1.1391, 1.36132, 1.58355, None], [-0.6483, -0.42608, -0.20386, 0.01836, 0.24059, 0.46281, 0.68503, 0.90725, 1.12948, 1.3517, None], [-0.56558, -0.34335, -0.12113, 0.10109, 0.32331, 0.54554, 0.76776, 0.98998, 1.2122, 1.43442, None], [-0.57482, -0.3526, -0.13038, 0.09185, 0.31407, 0.53629, 0.75851, 0.98074, 1.20296, 1.42518, None], [-0.4687, -0.24648, -0.02425, 0.19797, 0.42019, 0.64241, 0.86464, 1.08686, 1.30908, 1.5313, None]]


In [ ]:
def get_pred_from_causal(scm,values,cf_label, mapping_dict):
    #TODO change everything hard coded
    values['target']=cf_label
    def _get_noise_string(node):
        def _get_node_id(node):
            return node[1:]
        if not node[0] == "x":
            raise ValueError
        return "u" + _get_node_id(node)
    exogenous_variables = np.concatenate(
        [
            np.array(values[mapping_dict[node]]).reshape(-1, 1)
            for node in scm.get_topological_ordering("exogenous")
        ],
        axis=1,
    )
    exogenous_variables = pd.DataFrame(
        exogenous_variables, columns=scm.get_topological_ordering("exogenous")
    )

    endogenous_variables = exogenous_variables.copy()
    endogenous_variables = endogenous_variables.rename(
        columns=dict(
            zip(
                scm.get_topological_ordering("exogenous"),
                scm.get_topological_ordering("endogenous"),
            )
        )
    )
    # used later to make sure parents are populated when computing children
    endogenous_variables.loc[:] = np.nan
    for node in scm.get_topological_ordering("endogenous"):
        parents = scm.get_parents(node)
        if endogenous_variables.loc[:, list(parents)].isnull().values.any():
            raise ValueError(
                "parents in endogenous_variables should already be occupied"
            )
        endogenous_variables[node] = scm.structural_equations_np[node](
            exogenous_variables[_get_noise_string(node)],
            *[endogenous_variables[p] for p in parents],
        )
    labels=endogenous_variables['x7'][0]
    return labels

In [ ]:
mapping_dict={ 
    # Gender
      'u1': 'x1',
      # Age
      'u2': 'x2',
      # Education
      'u3': 'x3',
      # Loan amount
      'u4':'x4',
      # Loan duration
      'u5': 'x5',
      # Income
      'u6': 'x6',
      # Savings
      'u7':'x7',
}

In [ ]:
from carla.data import causal_model
from carla.evaluation import remove_nans
from carla.evaluation.api import Evaluation
class Sematic(Evaluation):
    """
    """

    def __init__(self, ml_model, causal_graph,mapping_dict):
        self.ml_model= ml_model
        self.causal_graph=causal_graph
        self.mapping_dict=mapping_dict
    def get_evaluation(self,factuals: np.ndarray, counterfactuals: np.ndarray):
        # generate data 
        cf_label=self.ml_model.predict(np.array(counterfactuals.values).reshape(1,-1))
        cf_label=np.argmax(cf_label)
        causal_label=get_pred_from_causal(self.causal_graph,counterfactuals,cf_label, self.mapping_dict)
        print(cf_label)
        if cf_label ==causal_label:
            return pd.DataFrame([[1]], columns=["semantic"])
        else: 
            return pd.DataFrame([[0]], columns=["semantic"])
''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
evaluation_measures = [
    Sematic(ml_model,scm)

]

results = benchmark.run_benchmark(evaluation_measures)


0
   semantic
0         0


'Histogram Features / true Positive / false Positive'